In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive 


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [2]:
cd /gdrive/My\ Drive/nn 

/gdrive/My Drive/nn


In [3]:
%tensorflow_version 1.x
import argparse
import os
import pickle
import sys

import numpy as np

np.random.seed(7)  # for reproducibility

import tensorflow as tf
tf.random.set_random_seed(5005)

from sklearn.model_selection import train_test_split, KFold

# tf.python.control_flow_ops = tf


from tensorflow.python.keras.models import Model, load_model
from tensorflow.python.keras.layers import Input
from tensorflow.python.keras.layers import Dense, Flatten, Dropout
from tensorflow.python.keras.layers.convolutional import Conv1D
from tensorflow.python.keras.layers.pooling import MaxPooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras.callbacks import ModelCheckpoint, EarlyStopping
import tensorflow.python.keras.backend as K

import matplotlib as mpl

mpl.use('Agg')
# from keras.utils.layer_utils import print_layer_shapes

import utils
sys.path.append(".")
from utils import load_data_merged
from utils import *

from tensorflow.python.keras.backend import set_session

model_path = "./models/final.h5"

CLASS_PARTIAL_LOSS = 0
CLASS_COMPLETELY_LOSS = 1 

_, _, _, _, x_test_seq, y_test = load_data_merged(r"dataset/solo_test_with_crc10_var_0.05.pkl", 150, False)

TensorFlow 1.x selected.


Using TensorFlow backend.


In [0]:
import re
ss_re = re.compile("[CG]CG[CG]")
ww_re = re.compile("[AT]CG[AT]")

In [0]:
def vecs2dna(seq_vecs):
    seqs = []
    # seq_vecs = np.reshape(seq_vecs, (seq_vecs.shape[0], 4, -1))
    for i in range(seq_vecs.shape[0]):
        seq_list = [''] * seq_vecs.shape[1]
        for j in range(seq_vecs.shape[1]):
            if seq_vecs[i, j,0] > 0:
                seq_list[j] = 'A'
            elif seq_vecs[i, j, 1] >0:
                seq_list[j] = 'C'
            elif seq_vecs[i, j, 2] >0:
                seq_list[j] = 'G'
            elif seq_vecs[i, j, 3] >0 :
                seq_list[j] = 'T'
            else:
                print("problem")

        seqs.append(''.join(seq_list))

    return seqs

def get_ex_ind(predict_label, thr=0.5):

  sess = tf.Session()
  tf.random.set_random_seed(5005)
  K.set_session(sess)

  session = K.get_session()
  init = tf.global_variables_initializer()
  session.run(init)

  with session.as_default():
      with session:
        model = load_model(model_path, custom_objects={'recall_TP': recall_TP,'recall_TN': recall_TN })
        pred = model.predict(x_test_seq)
        ind= []

        if predict_label ==CLASS_PARTIAL_LOSS:
          pred = 1 - pred

        for i in sorted(enumerate(pred), key=lambda x:x[1],reverse=True):
          if y_test[i[0]]==predict_label and pred[i[0]] > thr:
            ind.append(i[0])
        
  return ind

def get_prediction(data):

  sess = tf.Session()
  tf.random.set_random_seed(5005)
  K.set_session(sess)

  session = K.get_session()
  init = tf.global_variables_initializer()
  session.run(init)

  with session.as_default():
      with session:

        model = load_model(model_path, custom_objects={'recall_TP': recall_TP,'recall_TN': recall_TN })
        pred = model.predict(data)

  return pred

In [6]:
# Look at all the CL which are WW and change to SS
cl_ind = get_ex_ind(CLASS_COMPLETELY_LOSS)
cl_seq_array = np.array([x_test_seq[i] for i in cl_ind])
cl_seq = vecs2dna(cl_seq_array)
cl_ww_seq = [i for i in cl_seq if ww_re.search(i) != None]
cl_ss_seq = [i for i in cl_seq if ss_re.search(i) !=None]

cl_ww_to_ss = []
for seq in cl_ww_seq:
  new_seq = seq[:73] + "CCGC" + seq[77:]
  cl_ww_to_ss.append(new_seq)

  new_seq = seq[:73] + "GCGG" + seq[77:]
  cl_ww_to_ss.append(new_seq)

  new_seq = seq[:73] + "CCGG" + seq[77:]
  cl_ww_to_ss.append(new_seq)

  new_seq = seq[:73] + "GCGC" + seq[77:]
  cl_ww_to_ss.append(new_seq)

expected = len(cl_ww_to_ss)
cl_ww_to_ss = np.array([seq_to_mat(i) for i in cl_ww_to_ss])

pred = get_prediction(cl_ww_to_ss)
np.sum(pred>0.5) / expected * 100

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


19.563812600969303

In [0]:
# Look at all the PL which are SS and change to WW
pl_ind = get_ex_ind(CLASS_PARTIAL_LOSS)
pl_seq_array = np.array([x_test_seq[i] for i in pl_ind])

pl_seq = vecs2dna(pl_seq_array)
cl_ww_seq = [i for i in pl_seq if ww_re.search(i) != None]
ss_seq_seq = [i for i in pl_seq if ss_re.search(i) !=None]

pl_ss_to_ww = []
for seq in ss_seq_seq:
  new_seq = seq[:73] + "ACGA" + seq[77:]
  pl_ss_to_ww.append(new_seq)

  new_seq = seq[:73] + "ACGT" + seq[77:]
  pl_ss_to_ww.append(new_seq)

  new_seq = seq[:73] + "TCGT" + seq[77:]
  pl_ss_to_ww.append(new_seq)

  new_seq = seq[:73] + "TCGA" + seq[77:]
  pl_ss_to_ww.append(new_seq)

expected = len(pl_ss_to_ww)
pl_ww_to_ss = np.array([seq_to_mat(i) for i in pl_ss_to_ww])

pred = 1 - get_prediction(pl_ww_to_ss)
np.sum(pred>0.5) / expected * 100